# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Sep 21 2022 12:48PM,246949,10,0086000268,ISDIN Corporation,Released
1,Sep 21 2022 12:38PM,246946,20,8546457,"Exact-Rx, Inc.",Released
2,Sep 21 2022 12:38PM,246946,20,8546458,"Exact-Rx, Inc.",Released
3,Sep 21 2022 12:38PM,246946,20,8546460,"Exact-Rx, Inc.",Released
4,Sep 21 2022 12:38PM,246946,20,8546461,"Exact-Rx, Inc.",Released
5,Sep 21 2022 12:38PM,246946,20,8546462,"Exact-Rx, Inc.",Released
6,Sep 21 2022 12:38PM,246946,20,8546465,"Exact-Rx, Inc.",Released
7,Sep 21 2022 12:38PM,246946,20,8546467,"Exact-Rx, Inc.",Released
8,Sep 21 2022 12:38PM,246946,20,8546468,"Exact-Rx, Inc.",Released
9,Sep 21 2022 12:38PM,246946,20,8546471,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,246943,Released,1
34,246944,Released,5
35,246945,Executing,1
36,246946,Released,16
37,246949,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
246943,NaN,NaN,1.0
246944,NaN,NaN,5.0
246945,NaN,1.0,NaN
246946,NaN,NaN,16.0
246949,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246844,8.0,8.0,0.0
246864,5.0,0.0,1.0
246871,0.0,2.0,0.0
246872,0.0,0.0,1.0
246878,0.0,0.0,1.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
246844,8,8,0
246864,5,0,1
246871,0,2,0
246872,0,0,1
246878,0,0,1


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246844,8,8,0
1,246864,5,0,1
2,246871,0,2,0
3,246872,0,0,1
4,246878,0,0,1


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,246844,8,8,
1,246864,5,,1
2,246871,,2,
3,246872,,,1
4,246878,,,1


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Sep 21 2022 12:48PM,246949,10,ISDIN Corporation
1,Sep 21 2022 12:38PM,246946,20,"Exact-Rx, Inc."
17,Sep 21 2022 12:35PM,246945,22,"NBTY Global, Inc."
18,Sep 21 2022 12:33PM,246944,12,Hush Hush
23,Sep 21 2022 12:28PM,246942,19,"AdvaGen Pharma, Ltd"
45,Sep 21 2022 12:25PM,246943,10,"Nextsource Biotechnology, LLC"
46,Sep 21 2022 12:12PM,246940,10,Eywa Pharma Inc.
65,Sep 21 2022 12:04PM,246941,10,Eywa Pharma Inc.
66,Sep 21 2022 11:50AM,246939,12,"Zelenko Labs, LLC"
67,Sep 21 2022 11:36AM,246937,15,"Virtus Pharmaceuticals, LLC"


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Sep 21 2022 12:48PM,246949,10,ISDIN Corporation,,,1
1,Sep 21 2022 12:38PM,246946,20,"Exact-Rx, Inc.",,,16
2,Sep 21 2022 12:35PM,246945,22,"NBTY Global, Inc.",,1,
3,Sep 21 2022 12:33PM,246944,12,Hush Hush,,,5
4,Sep 21 2022 12:28PM,246942,19,"AdvaGen Pharma, Ltd",,4,18
5,Sep 21 2022 12:25PM,246943,10,"Nextsource Biotechnology, LLC",,,1
6,Sep 21 2022 12:12PM,246940,10,Eywa Pharma Inc.,,,19
7,Sep 21 2022 12:04PM,246941,10,Eywa Pharma Inc.,,1,
8,Sep 21 2022 11:50AM,246939,12,"Zelenko Labs, LLC",,,1
9,Sep 21 2022 11:36AM,246937,15,"Virtus Pharmaceuticals, LLC",,16,8


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 12:48PM,246949,10,ISDIN Corporation,1,,
1,Sep 21 2022 12:38PM,246946,20,"Exact-Rx, Inc.",16,,
2,Sep 21 2022 12:35PM,246945,22,"NBTY Global, Inc.",,1,
3,Sep 21 2022 12:33PM,246944,12,Hush Hush,5,,
4,Sep 21 2022 12:28PM,246942,19,"AdvaGen Pharma, Ltd",18,4,
5,Sep 21 2022 12:25PM,246943,10,"Nextsource Biotechnology, LLC",1,,
6,Sep 21 2022 12:12PM,246940,10,Eywa Pharma Inc.,19,,
7,Sep 21 2022 12:04PM,246941,10,Eywa Pharma Inc.,,1,
8,Sep 21 2022 11:50AM,246939,12,"Zelenko Labs, LLC",1,,
9,Sep 21 2022 11:36AM,246937,15,"Virtus Pharmaceuticals, LLC",8,16,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 12:48PM,246949,10,ISDIN Corporation,1,,
1,Sep 21 2022 12:38PM,246946,20,"Exact-Rx, Inc.",16,,
2,Sep 21 2022 12:35PM,246945,22,"NBTY Global, Inc.",,1,
3,Sep 21 2022 12:33PM,246944,12,Hush Hush,5,,
4,Sep 21 2022 12:28PM,246942,19,"AdvaGen Pharma, Ltd",18,4,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 12:48PM,246949,10,ISDIN Corporation,1.0,NaN,NaN
1,Sep 21 2022 12:38PM,246946,20,"Exact-Rx, Inc.",16.0,NaN,NaN
2,Sep 21 2022 12:35PM,246945,22,"NBTY Global, Inc.",NaN,1.0,NaN
3,Sep 21 2022 12:33PM,246944,12,Hush Hush,5.0,NaN,NaN
4,Sep 21 2022 12:28PM,246942,19,"AdvaGen Pharma, Ltd",18.0,4.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Sep 21 2022 12:48PM,246949,10,ISDIN Corporation,1.0,0.0,0.0
1,Sep 21 2022 12:38PM,246946,20,"Exact-Rx, Inc.",16.0,0.0,0.0
2,Sep 21 2022 12:35PM,246945,22,"NBTY Global, Inc.",0.0,1.0,0.0
3,Sep 21 2022 12:33PM,246944,12,Hush Hush,5.0,0.0,0.0
4,Sep 21 2022 12:28PM,246942,19,"AdvaGen Pharma, Ltd",18.0,4.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,3456953,84.0,10.0,6.0
12,740755,7.0,0.0,0.0
15,740771,27.0,32.0,0.0
16,740623,0.0,15.0,8.0
19,493875,19.0,4.0,0.0
20,740760,33.0,0.0,0.0
21,246878,1.0,0.0,0.0
22,246945,0.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,3456953,84.0,10.0,6.0
1,12,740755,7.0,0.0,0.0
2,15,740771,27.0,32.0,0.0
3,16,740623,0.0,15.0,8.0
4,19,493875,19.0,4.0,0.0
5,20,740760,33.0,0.0,0.0
6,21,246878,1.0,0.0,0.0
7,22,246945,0.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,84.0,10.0,6.0
1,12,7.0,0.0,0.0
2,15,27.0,32.0,0.0
3,16,0.0,15.0,8.0
4,19,19.0,4.0,0.0
5,20,33.0,0.0,0.0
6,21,1.0,0.0,0.0
7,22,0.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,84.0
1,12,Released,7.0
2,15,Released,27.0
3,16,Released,0.0
4,19,Released,19.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,20,21,22
Status,,,,,,,,
Completed,6.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0
Executing,10.0,0.0,32.0,15.0,4.0,0.0,0.0,1.0
Released,84.0,7.0,27.0,0.0,19.0,33.0,1.0,0.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,20,21,22
0,Completed,6.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0
1,Executing,10.0,0.0,32.0,15.0,4.0,0.0,0.0,1.0
2,Released,84.0,7.0,27.0,0.0,19.0,33.0,1.0,0.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,20,21,22
0,Completed,6.0,0.0,0.0,8.0,0.0,0.0,0.0,0.0
1,Executing,10.0,0.0,32.0,15.0,4.0,0.0,0.0,1.0
2,Released,84.0,7.0,27.0,0.0,19.0,33.0,1.0,0.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()